In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('/content/sample_data/Classification_train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29998,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_np=df.to_numpy()
df_np.shape

(30000, 785)

In [ ]:
X,y=df_np[:,1:],df_np[:,0:1]
X.shape,y.shape

((30000, 784), (30000, 1))

In [ ]:
def train_test_split(X,y,test_ratio=0.2,random_seed=None):
  if random_seed is not None:
    np.random.seed(random_seed)
  #Shuffle indices
  indices=np.arange(X.shape[0])
  np.random.shuffle(indices)
  #Calculate the number of samples for testing
  test_size=int(X.shape[0]*test_ratio)
  #Split the data
  test_indices=indices[:test_size]
  train_indices=indices[test_size:]
  X_train,X_test=X[train_indices],X[test_indices]
  y_train,y_test=y[train_indices],y[test_indices]
  return X_train,X_test,y_train,y_test

In [ ]:
#Split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_ratio=0.2,random_seed=42)

In [ ]:
X_train.shape

(24000, 784)

In [ ]:
def one_hot_encode(labels,num_classes):
  #Convert integer labels to one-hot encoding
  num_samples=len(labels)
  encoded_labels=np.zeros((num_samples,num_classes))
  for i in range(num_samples):
    encoded_labels[i,labels[i]]=1
  return encoded_labels

In [ ]:
def sigmoid(x):
    # Clip the values of x to prevent overflow
    clipped_x = np.clip(x, -500, 500)

    # Calculate the sigmoid function
    return 1 / (1 + np.exp(-clipped_x))

In [ ]:
def initialize_parameters(num_features,num_classes):
  #Initialize weights and bias
  W=np.zeros((num_features,num_classes))
  b=np.zeros((1,num_classes))
  return W,b

In [ ]:
def forward_propagation(X,W,b):
  #Forward propagation step
  Z=np.dot(X,W)+b
  A=sigmoid(Z)
  return A

In [ ]:
def compute_cost(A,Y):
  #Compute cross-entropyloss
  m=len(Y)
  cost=-1/m*np.sum(Y*np.log(A+1e-15)) #Add small epsilon to avoid log(0)
  return cost

In [ ]:
def backward_propagation(X,A,Y):
  #Backward propagation step
  m=len(Y)
  dZ=A-Y
  dW=1/m*np.dot(X.T,dZ)
  db=1/m*np.sum(dZ,axis=0,keepdims=True)
  return dW,db

In [ ]:
def update_parameters(W,b,dW,db,learning_rate):
  #Update weights and bias using gradient descent
  W-=learning_rate*dW
  b-=learning_rate*db
  return W,b

In [ ]:
def train(X,Y,num_classes,num_ietrations=1000,learning_rate=0.01):
  num_samples,num_features=X.shape
  W,b=initialize_parameters(num_features,num_classes)
  #One hot encode labels
  Y_encoded=one_hot_encode(Y,num_classes)
  for i in range(num_ietrations):
    #Forward propagation
    A=forward_propagation(X,W,b)
    #Compute cost
    cost=compute_cost(A,Y_encoded)
    #Backward propagation
    dW,db=backward_propagation(X,A,Y_encoded)
    #Update parameters
    W,b=update_parameters(W,b,dW,db,learning_rate)
    #Print cost every 100 iterations
    if i%100==0:
      print(f"Iteration {i},Cost: {cost}")
  return W,b

In [ ]:
def predict(X,W,b):
  #Make predictions using the trained model
  A=forward_propagation(X,W,b)
  return np.argmax(A,axis=1)

In [ ]:
num_classes=len(np.unique(y_train)) #Only the unique values of labels constitute the classes
num_iterations=1000
learning_rate=0.01

In [ ]:
def normalize_features(X):
  #Normalize features to have zero and unit variance
  epsilon=1e-8
  mean=np.mean(X,axis=0)
  std=np.std(X,axis=0)
  X_normalized=(X-mean)/(std+epsilon)
  return X_normalized,mean,std

In [ ]:
#Normalize features
X_normalized,mean,std=normalize_features(X_train)

In [ ]:
#Training the model
W,b=train(X_normalized,y_train,num_classes,num_iterations,learning_rate)

Iteration 0,Cost: 0.6931471805599431
Iteration 100,Cost: 0.1628631580299103
Iteration 200,Cost: 0.15080209825783564
Iteration 300,Cost: 0.15713871095054183
Iteration 400,Cost: 0.1664286059496804
Iteration 500,Cost: 0.17540969473782372
Iteration 600,Cost: 0.18320203208115
Iteration 700,Cost: 0.18965047261482046
Iteration 800,Cost: 0.19484761618449017
Iteration 900,Cost: 0.19896173612465493


In [ ]:
#Normalize test features
X_normalized_test,mean,std=normalize_features(X_test)

In [ ]:
#Make predictions
predictions=predict(X_normalized_test,W,b)

In [ ]:
predictions

array([3, 8, 6, ..., 1, 1, 3])

In [ ]:
#Evaluate accuracy
accuracy=np.mean(predictions==y_test)
print(f"Accuracy on Test Data: {accuracy * 100:.2f}%")

Accuracy on Test Data: 10.02%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score #For assessing my model I have imported sklearn module

# Add zero_division=1 to handle the warning
precision = precision_score(y_test,predictions, average='weighted', zero_division=1)
recall = recall_score(y_test,predictions, average='weighted', zero_division=1)
f1 = f1_score(y_test, predictions, average='weighted', zero_division=1)

In [ ]:
precision

0.9262072353761356

In [ ]:
recall

0.9263333333333333

In [ ]:
f1

0.9257406172437674

On the test dataset


In [ ]:
pf=pd.read_csv('/content/sample_data/Classification_test.csv')
pf

,ID,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,15795,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5390,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11964,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11284,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11260,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,18563,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,634,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,10057,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pf_np=pf.to_numpy()
pf_np.shape

(10000, 785)

In [ ]:
P_test,q_test=df_np[:,1:],df_np[:,0:1]
P_test.shape,q_test.shape

((30000, 784), (30000, 1))

In [ ]:
#Normalize features
P_test_normalized,mean,std=normalize_features(P_test)

In [ ]:
#Make predictions
predictions_test=predict(P_test_normalized,W,b)

In [ ]:
#Evaluate accuracy
accuracy=np.mean(predictions_test==q_test)
print(f"Accuracy on Test Data: {accuracy * 100:.2f}%")

Accuracy on Test Data: 10.01%


In [ ]:
#Add zero_division=1 to handle the warning
precision = precision_score(q_test,predictions_test, average='weighted', zero_division=1)
recall = recall_score(q_test,predictions_test, average='weighted', zero_division=1)
f1 = f1_score(q_test, predictions_test, average='weighted', zero_division=1)

In [ ]:
precision

0.9308908447584754

In [ ]:
recall

0.9309

In [ ]:
f1

0.9305710069833564